In [20]:
import nltk
import re
from nltk.corpus import wordnet as wn
from itertools import product
from nltk.corpus import wordnet_ic


def get_sim_score(word_1, word_2, info_content):
    """ 
    Calculate the highest path similarity among all pairs. 
    """

    if word_1 == word_2:
        return 1
    else:
        max_sim = 0.0
        synsets_1 = wn.synsets(word_1)
        synsets_2 = wn.synsets(word_2)
        if synsets_1 and synsets_2:
            for synset_1, synset_2 in product(synsets_1, synsets_2):
                try:
                    sim = wn.lin_similarity(synset_1, synset_2, info_content)
                    #sim = wn.jcn_similarity(synset_1, synset_2, info_content)
                    #sim = wn.wup_similarity(synset_1, synset_2)
                    if sim > max_sim:
                        max_sim = sim
                except:
                    continue

            return max_sim
        return max_sim

def remove_dup(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]


def mySim(text1, text2, sigma=0.85, w=0.3, corpus='ic-brown-resnik.dat'):
    # set stop words
    stopwords = nltk.corpus.stopwords.words('english')
    # set variables
    x = []
    y = []
    dic = {}
    info_content = wordnet_ic.ic(corpus)

    # clean raw text
    text1 = re.sub('[^a-zA-Z]', ' ', text1).lower()
    text2 = re.sub('[^a-zA-Z]', ' ', text2).lower()

    # tokenize inputs into vectors
    token_p = nltk.word_tokenize(text1, language='english')
    token_r = nltk.word_tokenize(text2, language='english')

    concept_p = [words for words in token_p if words not in stopwords]
    concept_r = [words for words in token_r if words not in stopwords]

    
    
    # pos the tokens and n-grams
    pos_p = [word for word, tag in nltk.pos_tag(concept_p) if tag.startswith('NN') or tag.startswith('JJ')]
    pos_p.extend([' '.join(words).strip() for words in nltk.ngrams(pos_p, 2)])
    
    pos_r = [word for word, tag in nltk.pos_tag(concept_r) if tag.startswith('NN') or tag.startswith('JJ')]
    pos_r.extend([' '.join(words).strip() for words in nltk.ngrams(pos_r, 2)])
    
    #print(pos_p, pos_r)
    
 
    total_sim = 0
    total_count = 0
 
    for w1, w2 in product(pos_p, pos_r):
        sim = 0
        w1_set = set(w1.split())
        w2_set = set(w2.split())
        
        count = 0
        for t1, t2 in product(w1_set, w2_set):
            sim += get_sim_score(t1, t2, info_content)
            count += 1
            
            
        if sim/count >= sigma:
            total_sim += sim
            total_count += 1
            
            x.append(w1)
            y.append(w2)
            dic[w1] = w2            
            #print(sim, '\t\t', w1_set, '|',  w2_set)

    concept_x = [words for words in pos_p if words in x]
    concept_y = [words for words in pos_r if words in y]

    print(set(x), set(y))
    
    if total_count:
        return  total_sim/total_count
    else:
        return 0


    
def concept_matching(concepts, sentence, sigma=0.85, w=0.3, corpus='ic-brown-resnik.dat'):
    # set stop words
    stopwords = nltk.corpus.stopwords.words('english')
    # set variables
    x = []
    y = []
    dic = {}
    info_content = wordnet_ic.ic(corpus)

    # clean raw text
    concepts = [re.sub('[^a-zA-Z]', ' ', word).lower() for word in concepts]
    context = re.sub('[^a-zA-Z]', ' ', sentence).lower()

    # tokenize inputs into vectors
    tokens = nltk.word_tokenize(context, language='english')
    
    # remove stopwords
    concept_p = [words for words in tokens if words not in stopwords]

    
    
    # pos the tokens and n-grams
    pos_p = [word for word, tag in nltk.pos_tag(concept_p) if tag.startswith('NN') or tag.startswith('JJ')]
    pos_p.extend([' '.join(words).strip() for words in nltk.ngrams(pos_p, 2)])
    
    #print(pos_p)

    count = 0
    total_sim = 0
    
    for w1, w2 in product(concepts, pos_p):
        sim = 0
        w1_set = set(w1.split())
        w2_set = set(w2.split())
        for t1, t2 in product(w1_set, w2_set):
            sim += get_sim_score(t1, t2, info_content)
            #print(sim, '\t\t', t1, t2, '\t\t', w1_set, '|',  w2_set)
            
            
        if sim >= sigma:

            count += 1
            total_sim += sim
            
            #x.append(w1)
            #y.append(w2)
            #dic[w1] = w2
            #print(sim, '\t\t', w1_set, '|',  w2_set)

    #concept_x = [words for words in concepts if words in x]
    #concept_y = [words for words in pos_p if words in y]

    #print(concept_x, concept_y)
    if count:
        return  total_sim/count
    else:
        return 0

In [13]:
import pandas as pd

df = pd.read_csv('bhutan_sentence.csv')[['sector', 'sentence']]
goals = pd.read_csv('SDG_Goal.csv')
goals.columns = ['goal', 'text']

goals.head(10)

,goal,text
0,1.1,"By 2030, eradicate extreme povert yfor all peo..."
1,1.2,"By 2030, reduce at least by half the proportio..."
2,1.3,Implement nationally appropriate social protec...
3,1.4,"By 2030, ensure that all men and women, in par..."
4,1.5,"By 2030, build the resilience of the poor and ..."
5,2.1,"By 2030, end hunger and ensure access by all p..."
6,2.2,"By 2030, end all forms of malnutrition, includ..."
7,2.3,"By 2030, double the agricultural productivity ..."
8,2.4,"By 2030, ensure sustainable food production sy..."
9,2.5,"By 2020, maintain the genetic diversity of see..."


In [14]:
count = 1

for ind in goals.index.values:
    target = goals.loc[ind, 'text']
    goal = goals.loc[ind, 'goal']
    for i in df.index.values:
        df.loc[i , goal] = mySim(df.loc[i, 'sentence'], target)
    
    count += 1
    if count > 5:
        break

In [24]:
df.to_csv('test0423_3.csv')

In [79]:
concept_matching(['pingpong', 'best players'], 'Many consider Malin as the best player in PingPong history', 1 )

3.75e+299

In [23]:
mySim('Many consider Malin as the best player in PingPong history', 'Malin is one of the best PingPong players', 0.85)

{'pingpong', 'player', 'best', 'malin'} {'pingpong', 'players', 'best', 'malin'}


1.0

In [40]:
from nltk import tokenize
with open('/Users/maxwelllee54/GitHubs/Rapid_Assessment_Tools/Bhutan_Input.txt') as f:
    data = f.read()
    sen = tokenize.sent_tokenize(data)

In [7]:
goal1_1 = ['extreme poverty']

In [8]:
target1_1 = 'By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day'

In [9]:
test = 'An educated citizenry that is able to understand, translate and act upon information, laws, rights and responsibilities is critical for ensuring a "vibrant democracy" and effective governance.'

In [10]:
g_test = 'By 2017, the proportion of severely poor individuals has dropped from 15.8% in 2009/10 to below 10%'

In [52]:
concept_matching(goal1_1, test, sigma=0.85)

1.3570191713048856

In [19]:
brown_ic = wordnet_ic.ic('ic-brown-resnik.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

In [11]:
mySim(target1_1, test, sigma=0.85)

1.0

In [13]:
from nltk.corpus import wordnet as wn
from itertools import product

synsets_1 = wn.synsets('poverty')
synsets_2 = wn.synsets('individuals')
if synsets_1 and synsets_2:
    for synset_1, synset_2 in product(synsets_1, synsets_2):
        try:
            sim = wn.wup_similarity(synset_1, synset_2)
            print(sim, synset_1, synset_2)
        except:
            continue

0.18181818181818182 Synset('poverty.n.01') Synset('person.n.01')
0.14285714285714285 Synset('poverty.n.01') Synset('individual.n.02')


In [80]:
import pandas as pd 
result = pd.DataFrame()
i = 0

for s in sen: 
    if concept_matching(goal1_1, s, sigma=0.4):
        result.loc[i, 'text'] = s
        result.loc[i, 'score'] = concept_matching(goal1_1, s, sigma=0.4)
        i +=1
        print(s)

﻿5.1 Education Sector

The single most important factor towards achieving the socio-economic development goals will be the success of the education sector in enhancing the social, economic, cultural, environmental, and political capabilities of our most important asset, our people.


Education virtually impacts all dimensions of development and is critical for ensuring sustainable poverty alleviation which can only come through the empowerment of our people, particularly the remote and most vulnerable sections of our population.


At the end of the cycle (Class X) there is a national board examination, Bhutan Certificate for Secondary Education (BCSE).


Programmes offered vary from 6 months to 2 years at the certificate level in courses ranging from electrical, mechanical, construction engineering, auto-mechanics, driving, computer hardware, carpentry, furniture making.


Tertiary education in Bhutan comprises of three years of undergraduate studies provided by the Royal University of Bhutan (RUB) through its 10 constituent colleges located in various parts of the country and one private college, the Royal Thimphu College.


...

To strengthen the process of decentralization, administrative and fiscal devolution of authority alone is not sufficient but of equal importance is to build the capacity of those at the grassroots to plan, implement and monitor development activities.


Human resource strengthening at local levels both in terms of numbers and technical skills will be critical.


The rapid growth of civil service, which averaged 4 percent per annum during the last five years, and high costs, amounting to about 45 percent of the total recurrent expenditures, is a concern.


The turn-around time for many of the public services to its citizens, businesses and intra government services remain high and inefficient.


The major areas of concerns are the misuse of authority, misuse of government property, embezzlement and nepotism.


In [84]:
result.to_csv('test.csv')

In [104]:
import pandas as pd 
result2 = pd.DataFrame()
i = 0

for s in sen: 
    if mySim(target1_1, s, sigma=0.4):
        result.loc[i, 'text'] = s
        result.loc[i, 'score'] = concept_matching(goal1_1, s, sigma=0.4)
        i += 1
        print(s)
result2.to_csv('test2.csv')

﻿5.1 Education Sector

The single most important factor towards achieving the socio-economic development goals will be the success of the education sector in enhancing the social, economic, cultural, environmental, and political capabilities of our most important asset, our people.


Education virtually impacts all dimensions of development and is critical for ensuring sustainable poverty alleviation which can only come through the empowerment of our people, particularly the remote and most vulnerable sections of our population.


An educated citizenry that is able to understand, translate and act upon information, laws, rights and responsibilities is critical for ensuring a "vibrant democracy" and effective governance.


Similarly, the realization of our vision of a "knowledge based society" is absolutely dependent on a technically skilled workforce with competencies benchmarked to leading industry standards and capabilities to think globally but act locally.


The responsibility for the administration of education in Bhutan is shared amongst the Ministry of Education (MoE), the Ministry of Labour and Human Resources (MoLHR), the Royal University of Bhutan (RUB), and the local governments.


...

5.18.4 Strategies

•	Education, awareness and capacity building will be one of the key strategies to enhance political participation, particularly the youth and women.


•	The capacity of the local governments will be strengthened through training, deployment of adequate number of technically qualified personnel and ensuring minimum facilities are available.


•	A legislation to ensure quota for women in elected offices including the parliament and local government bodies will be drafted and presented to the Parliament.


•	Civil service rules and regulations will be reviewed and revised to make civil service more efficient.


•	The government will ensure full implementation of the National Anti-corruption strategy and agencies combating corruption will be strengthened and equipped with adequate resources.


In [105]:
result.to_csv('test2.csv')

In [ ]:
'''
    count = len(concept_x)
    total = 0
    
    m = len(concept_p)
    n = len(concept_r)

    if count == 0:
        sim_score_0 = 0

    elif count % 2 == 0 or count == 1:
        for position, word in enumerate(concept_x):
            total += abs(position - concept_y.index(dic[word]))

        sim_score_0 = 1 - 2 * total / count ** 2

    else:
        for position, word in enumerate(concept_x):
            total += abs(position - concept_y.index(dic[word]))

        sim_score_0 = 1 - 2 * total / (count ** 2 - 1)

    S = ((m + n) / (2 * m * n)) * (count * (1 - w * (1 - sim_score_0)))

    return S
'''